<a href="https://colab.research.google.com/github/MateoPeri/Helyx-Submission/blob/master/Helyx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import numpy as np
from scipy import spatial
import pandas as pd
import glob
!pip install utm
import utm

In [2]:
!rm -rf '/content/data'
!mkdir data
PATH = '/content/data/'
!wget -P {PATH} https://raw.githubusercontent.com/MateoPeri/Helyx-Submission/master/data.zip
!unzip {PATH}data.zip
!wget -P {PATH} https://raw.githubusercontent.com/MateoPeri/Helyx-Submission/master/estaciones.csv

--2020-04-11 23:55:36--  https://raw.githubusercontent.com/MateoPeri/Helyx-Submission/master/data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1642601 (1.6M) [application/zip]
Saving to: ‘/content/data/data.zip’

data.zip            100%[===================>]   1.57M  --.-KB/s    in 0.1s    

2020-04-11 23:55:36 (13.1 MB/s) - ‘/content/data/data.zip’ saved [1642601/1642601]

Archive:  /content/data/data.zip
  inflating: data/AL31.csv           
  inflating: data/AL62.csv           
  inflating: data/AL91.csv           
  inflating: data/CA12.csv           
  inflating: data/CA21.csv           
  inflating: data/CA52.csv           
  inflating: data/CA73.csv           
  inflating: data/CA91.csv           
  inflating: data/CI32.csv           
  inflating: 

In [79]:
pattern = 'data/*.csv'
csv_files = glob.glob(pattern)

frames = []

for csv in csv_files:
  if csv == 'data/estaciones.csv':
    continue
  df = pd.read_csv(csv, parse_dates=True, sep=';', escapechar='\\', encoding='ISO-8859-1', decimal=',', thousands='.')
  frames.append(df)

df = pd.concat(frames, ignore_index=True)

df.head()

,ESTACION,MUNICIPIO,PARAJE,HORAS,FECHA,HSOL (h),RADMAX (w/m2),RADMED (w/m2),Unnamed: 8
0,MU31,Murcia,La Vereda,24.0,01/01/00,7.0,482.0,111.08,NaN
1,MU31,Murcia,La Vereda,24.0,02/01/00,7.0,485.0,111.88,NaN
2,MU31,Murcia,La Vereda,24.0,03/01/00,6.0,545.0,77.83,NaN
3,MU31,Murcia,La Vereda,24.0,04/01/00,6.0,630.0,82.21,NaN
4,MU31,Murcia,La Vereda,24.0,05/01/00,7.0,481.0,109.04,NaN


In [78]:
df.dtypes

ESTACION           object
MUNICIPIO          object
PARAJE             object
HORAS             float64
FECHA              object
HSOL  (h)         float64
RADMAX  (w/m2)    float64
RADMED  (w/m2)    float64
Unnamed: 8        float64
dtype: object

In [80]:
df.FECHA = pd.to_datetime(df.FECHA)
df = df.drop(['Unnamed: 8', 'HORAS'], axis=1)
df.rename(columns={'HSOL  (h)': 'HSOL', 'RADMAX  (w/m2)': 'RADMAX', 'RADMED  (w/m2)': 'RADMED'}, inplace=True)

df.head()

,ESTACION,MUNICIPIO,PARAJE,FECHA,HSOL,RADMAX,RADMED
0,MU31,Murcia,La Vereda,2000-01-01,7.0,482.0,111.08
1,MU31,Murcia,La Vereda,2000-02-01,7.0,485.0,111.88
2,MU31,Murcia,La Vereda,2000-03-01,6.0,545.0,77.83
3,MU31,Murcia,La Vereda,2000-04-01,6.0,630.0,82.21
4,MU31,Murcia,La Vereda,2000-05-01,7.0,481.0,109.04


# Preprocessing

In [81]:
stations = pd.read_csv('data/estaciones.csv', sep=';')
stations['Utmy'], stations['Utmx'] = utm.to_latlon(stations['Utmx'], stations['Utmy'], zone_number=30, zone_letter="S")
stations.rename(columns={'Estación': 'ESTACION', 'Utmy': 'LAT', 'Utmx': 'LONG'}, inplace=True)
stations.head()

,ESTACION,LAT,LONG
0,AL31,37.734317,-1.511796
1,AL62,37.562186,-1.400979
2,CA12,37.690459,-0.949582
3,CA21,37.833234,-1.121357
4,CA52,37.678868,-1.069549


In [83]:
result = pd.merge(df, stations, on='ESTACION')
result.head()

,ESTACION,MUNICIPIO,PARAJE,FECHA,HSOL,RADMAX,RADMED,LAT,LONG
0,MU31,Murcia,La Vereda,2000-01-01,7.0,482.0,111.08,37.900056,-1.266895
1,MU31,Murcia,La Vereda,2000-02-01,7.0,485.0,111.88,37.900056,-1.266895
2,MU31,Murcia,La Vereda,2000-03-01,6.0,545.0,77.83,37.900056,-1.266895
3,MU31,Murcia,La Vereda,2000-04-01,6.0,630.0,82.21,37.900056,-1.266895
4,MU31,Murcia,La Vereda,2000-05-01,7.0,481.0,109.04,37.900056,-1.266895


# TODO



*   Missings
*   Entrenar (X=[FECHA, HSOL, RADMAX, RADMED, LAT, LONG])
*   Validar

